In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [58]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)
query = '''
    select film_id, inventory_id, rental_date, store_id, staff_id, 
    rental_rate, rental_duration, length, rating, c.name
    from (
        select *
        from sakila.rental
        where year(rental_date)<=2005) re
    right join sakila.inventory
    using (inventory_id)
    right join  sakila.film
    using (film_id)
    join sakila.film_category
    using (film_id)
    join sakila.category c
    using (category_id);
'''

data = pd.read_sql_query(query, engine)
data.isna().sum()

film_id             0
inventory_id       42
rental_date        43
store_id           42
staff_id           43
rental_rate         0
rental_duration     0
length              0
rating              0
name                0
dtype: int64

In [10]:
import datetime as dt

In [30]:
display(data.rating.value_counts())
display(data.name.value_counts())
data['ryear']=data.rental_date.apply(lambda x: x.year)
data['rmonth']=data.rental_date.apply(lambda x: x.month)
display(data.ryear.value_counts())
display(data.rmonth.value_counts())
display(data.head())
print(data.dtypes)


PG-13    3553
NC-17    3257
PG       3190
R        3158
G        2747
Name: rating, dtype: int64

Sports         1165
Animation      1147
Action         1098
Sci-Fi         1095
Family         1085
Drama          1054
Documentary    1050
Foreign        1028
Games           958
Children        941
Comedy          934
Classics        933
New             930
Horror          837
Travel          831
Music           819
Name: name, dtype: int64

2005.0    15862
Name: ryear, dtype: int64

7.0    6709
8.0    5686
6.0    2311
5.0    1156
Name: rmonth, dtype: int64

,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name,ryear,rmonth
0,19,93.0,2005-08-02 01:16:59,1.0,2.0,0.99,6,113,PG,Action,2005.0,8.0
1,19,93.0,2005-08-18 04:26:54,1.0,2.0,0.99,6,113,PG,Action,2005.0,8.0
2,19,94.0,2005-06-20 20:35:28,1.0,1.0,0.99,6,113,PG,Action,2005.0,6.0
3,19,94.0,2005-07-09 05:01:58,1.0,2.0,0.99,6,113,PG,Action,2005.0,7.0
4,19,94.0,2005-07-30 08:02:39,1.0,1.0,0.99,6,113,PG,Action,2005.0,7.0


film_id                     int64
inventory_id              float64
rental_date        datetime64[ns]
store_id                  float64
staff_id                  float64
rental_rate               float64
rental_duration             int64
length                      int64
rating                     object
name                       object
ryear                     float64
rmonth                    float64
dtype: object


There exist many rows with the same film_id. It would be much simpler if I have a own list which only includes films 
and one list for the inventory.

In [60]:
rental=data
query2='''
    select film_id, rental_rate, rental_duration, length, rating, c.name
    from sakila.film
    join sakila.film_category
    using (film_id)
    join sakila.category c
    using (category_id);
'''
film = pd.read_sql_query(query2, engine)

query3='''
    select *
    from sakila.inventory;
'''
inventory = pd.read_sql_query(query3, engine)

In [39]:
display(film)
display(inventory)


,film_id,rental_rate,rental_duration,length,rating,name
0,19,0.99,6,113,PG,Action
1,21,4.99,3,129,R,Action
2,29,2.99,5,168,NC-17,Action
3,38,0.99,6,68,NC-17,Action
4,56,2.99,6,129,G,Action
...,...,...,...,...,...,...
995,931,0.99,7,48,PG-13,Travel
996,977,2.99,3,85,R,Travel
997,981,0.99,7,55,NC-17,Travel
998,988,2.99,7,139,R,Travel


,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17
...,...,...,...,...
4576,4577,1000,1,2006-02-15 05:09:17
4577,4578,1000,2,2006-02-15 05:09:17
4578,4579,1000,2,2006-02-15 05:09:17
4579,4580,1000,2,2006-02-15 05:09:17


1000

In [55]:
inventory2=pd.crosstab(inventory.film_id,inventory.store_id)
film2=pd.merge(left=film,
                right=inventory2,
                how='left',
                left_on='film_id',
                right_on='film_id')
film2=film2.rename(columns={1:'istore_1',2:'istore_2'})
film2=film2.fillna(0)
film2

,film_id,rental_rate,rental_duration,length,rating,name,istore_1,istore_2
0,19,0.99,6,113,PG,Action,4.0,2.0
1,21,4.99,3,129,R,Action,2.0,4.0
2,29,2.99,5,168,NC-17,Action,2.0,0.0
3,38,0.99,6,68,NC-17,Action,0.0,0.0
4,56,2.99,6,129,G,Action,3.0,2.0
...,...,...,...,...,...,...,...,...
995,931,0.99,7,48,PG-13,Travel,0.0,3.0
996,977,2.99,3,85,R,Travel,0.0,3.0
997,981,0.99,7,55,NC-17,Travel,3.0,3.0
998,988,2.99,7,139,R,Travel,3.0,2.0


nan inside the rental  are not rented films=> can be droped now

In [61]:
rental=rental.dropna(axis=0)
rental

,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name
0,19,93.0,2005-08-02 01:16:59,1.0,2.0,0.99,6,113,PG,Action
1,19,93.0,2005-08-18 04:26:54,1.0,2.0,0.99,6,113,PG,Action
2,19,94.0,2005-06-20 20:35:28,1.0,1.0,0.99,6,113,PG,Action
3,19,94.0,2005-07-09 05:01:58,1.0,2.0,0.99,6,113,PG,Action
4,19,94.0,2005-07-30 08:02:39,1.0,1.0,0.99,6,113,PG,Action
...,...,...,...,...,...,...,...,...,...,...
15900,989,4535.0,2005-05-30 18:45:17,2.0,1.0,4.99,4,74,R,Travel
15901,989,4535.0,2005-06-15 09:03:52,2.0,1.0,4.99,4,74,R,Travel
15902,989,4535.0,2005-07-07 06:38:31,2.0,1.0,4.99,4,74,R,Travel
15903,989,4535.0,2005-07-30 00:12:41,2.0,1.0,4.99,4,74,R,Travel


In [69]:
rentalst=pd.crosstab(rental.film_id,rental.store_id)
display(rentalst)
rental['month']=rental.rental_date.apply(lambda x: x.strftime('%b'))
rentalmo=pd.crosstab(rental.film_id,rental.month)
display(rentalmo)
rentalem=pd.crosstab(rental.film_id,rental.staff_id)
display(rentalem)

store_id,1.0,2.0
film_id,,
1,12,11
2,0,6
3,0,12
4,13,9
5,0,11
...,...,...
996,7,0
997,6,0
998,0,8


C:\Users\lahrm\AppData\Local\Temp/ipykernel_12728/3876040618.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental['month']=rental.rental_date.apply(lambda x: x.strftime('%b'))


month,Aug,Jul,Jun,May
film_id,,,,
1,9,9,3,2
2,4,2,0,0
3,6,4,1,1
4,6,12,2,2
5,3,5,2,1
...,...,...,...,...
996,3,3,1,0
997,3,2,1,0
998,2,3,2,1


staff_id,1.0,2.0
film_id,,
1,14,9
2,3,3
3,7,5
4,12,10
5,8,3
...,...,...
996,4,3
997,4,2
998,2,6


In [119]:
rental.month.value_counts()

Jul    6709
Aug    5686
Jun    2311
May    1156
Name: month, dtype: int64

In [72]:
rentalst=rentalst.rename(columns={1:'rstore_1',2:'rstore_2'})
rentalem=rentalem.rename(columns={1:'employee_1',2:'employee_2'})

In [76]:
film3=pd.merge(left=film2,
                right=rentalst,
                how='left',
                left_on='film_id',
                right_on='film_id')
film3=pd.merge(left=film3,
                right=rentalem,
                how='left',
                left_on='film_id',
                right_on='film_id')
film3=pd.merge(left=film3,
                right=rentalmo,
                how='left',
                left_on='film_id',
                right_on='film_id')

film3=film3.fillna(0)
film3

,film_id,rental_rate,rental_duration,length,rating,name,istore_1,istore_2,rstore_1,rstore_2,employee_1,employee_2,Aug,Jul,Jun,May
0,19,0.99,6,113,PG,Action,4.0,2.0,13.0,7.0,12.0,8.0,7.0,8.0,4.0,1.0
1,21,4.99,3,129,R,Action,2.0,4.0,10.0,11.0,10.0,11.0,8.0,8.0,3.0,2.0
2,29,2.99,5,168,NC-17,Action,2.0,0.0,10.0,0.0,6.0,4.0,2.0,4.0,2.0,2.0
3,38,0.99,6,68,NC-17,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,56,2.99,6,129,G,Action,3.0,2.0,11.0,7.0,7.0,11.0,6.0,8.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,931,0.99,7,48,PG-13,Travel,0.0,3.0,0.0,12.0,5.0,7.0,5.0,3.0,2.0,2.0
996,977,2.99,3,85,R,Travel,0.0,3.0,0.0,11.0,5.0,6.0,2.0,6.0,3.0,0.0
997,981,0.99,7,55,NC-17,Travel,3.0,3.0,10.0,11.0,8.0,13.0,8.0,9.0,4.0,0.0
998,988,2.99,7,139,R,Travel,3.0,2.0,8.0,7.0,7.0,8.0,7.0,6.0,2.0,0.0


Film_id was used for the combinations of the rows and can be droped now

In [95]:
film3=film3.drop('film_id',axis=1)

KeyError: "['film_id'] not found in axis"

The Aug column will be used as y and transformed in a boolean for item >0

In [99]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

y=pd.DataFrame(film3.Aug >0)
X=film3.drop('Aug',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_cat=X_train.select_dtypes(include=object)
X_train_num=X_train.select_dtypes(include=np.number)

In [103]:

X_train_cat=pd.get_dummies(X_train_cat,drop_first=True)
transformer = MinMaxScaler().fit(X_train_num)
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized)



In [115]:
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)
from sklearn.linear_model import LogisticRegression
classification1 = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train_transformed, y_train)
X_train_transformed = np.concatenate([X_train_norm, X_train_cat], axis=1)
from sklearn.linear_model import LogisticRegression
classification2 = LogisticRegression(random_state=0, solver='saga').fit(X_train_transformed, y_train)

C:\Users\lahrm\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\lahrm\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [116]:
X_test_num = X_test.select_dtypes(include = np.number)
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized)
X_test_categorical = X_test.select_dtypes(include = np.object)
X_test_cat = pd.get_dummies(X_test_categorical,drop_first=True)
display(list(zip(list(X_train_cat.columns),list(X_test_cat.columns))))

C:\Users\lahrm\AppData\Local\Temp/ipykernel_12728/2466316024.py:4: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test_categorical = X_test.select_dtypes(include = np.object)


[('rating_NC-17', 'rating_NC-17'),
 ('rating_PG', 'rating_PG'),
 ('rating_PG-13', 'rating_PG-13'),
 ('rating_R', 'rating_R'),
 ('name_Animation', 'name_Animation'),
 ('name_Children', 'name_Children'),
 ('name_Classics', 'name_Classics'),
 ('name_Comedy', 'name_Comedy'),
 ('name_Documentary', 'name_Documentary'),
 ('name_Drama', 'name_Drama'),
 ('name_Family', 'name_Family'),
 ('name_Foreign', 'name_Foreign'),
 ('name_Games', 'name_Games'),
 ('name_Horror', 'name_Horror'),
 ('name_Music', 'name_Music'),
 ('name_New', 'name_New'),
 ('name_Sci-Fi', 'name_Sci-Fi'),
 ('name_Sports', 'name_Sports'),
 ('name_Travel', 'name_Travel')]

In [117]:
X_test_transformed = np.concatenate([X_test_norm, X_test_cat], axis=1)
predictions = classification1.predict(X_test_transformed)
print(classification1.score(X_test_transformed, y_test))

from sklearn.metrics import confusion_matrix
display(confusion_matrix(y_test, predictions))

0.985


array([[ 12,   3],
       [  0, 185]], dtype=int64)

In [118]:
predictions = classification2.predict(X_test_transformed)
print(classification2.score(X_test_transformed, y_test))

from sklearn.metrics import confusion_matrix
display(confusion_matrix(y_test, predictions))

0.985


array([[ 12,   3],
       [  0, 185]], dtype=int64)

Solution could be cheated because the counts for rstores and rstaff also includes films which were rented in August

Next try only use data from rental from previos month to predict current month

In [122]:
rental_may=rental[rental.month=='May']
rental_jun=rental[rental.month=='Jun']
rental_jul=rental[rental.month=='Jul']
rental_aug=rental[rental.month=='Aug']
display(rental_may)
display(rental_jun)
display(rental_jul)
display(rental_aug)

,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name,month
6,19,95.0,2005-05-28 12:08:37,1.0,2.0,0.99,6,113,PG,Action,May
20,21,102.0,2005-05-26 08:04:38,1.0,2.0,4.99,3,129,R,Action,May
25,21,103.0,2005-05-25 21:58:58,1.0,1.0,4.99,3,129,R,Action,May
41,29,142.0,2005-05-31 16:48:43,1.0,2.0,2.99,5,168,NC-17,Action,May
46,29,143.0,2005-05-31 19:14:15,1.0,2.0,2.99,5,168,NC-17,Action,May
...,...,...,...,...,...,...,...,...,...,...,...
15810,914,4204.0,2005-05-28 18:25:02,2.0,1.0,2.99,6,61,PG,Travel,May
15821,931,4278.0,2005-05-25 19:30:46,2.0,2.0,0.99,7,48,PG-13,Travel,May
15826,931,4279.0,2005-05-30 15:05:47,2.0,2.0,0.99,7,48,PG-13,Travel,May
15895,989,4534.0,2005-05-28 10:12:41,2.0,2.0,4.99,4,74,R,Travel,May


,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name,month
2,19,94.0,2005-06-20 20:35:28,1.0,1.0,0.99,6,113,PG,Action,Jun
7,19,95.0,2005-06-20 00:08:38,1.0,1.0,0.99,6,113,PG,Action,Jun
13,19,97.0,2005-06-16 15:51:52,2.0,1.0,0.99,6,113,PG,Action,Jun
16,19,98.0,2005-06-19 10:20:09,2.0,1.0,0.99,6,113,PG,Action,Jun
21,21,102.0,2005-06-15 19:49:41,1.0,2.0,4.99,3,129,R,Action,Jun
...,...,...,...,...,...,...,...,...,...,...,...
15880,989,4530.0,2005-06-15 06:21:30,1.0,2.0,4.99,4,74,R,Travel,Jun
15884,989,4531.0,2005-06-19 11:12:35,1.0,2.0,4.99,4,74,R,Travel,Jun
15888,989,4532.0,2005-06-21 11:38:45,1.0,1.0,4.99,4,74,R,Travel,Jun
15896,989,4534.0,2005-06-20 08:06:18,2.0,1.0,4.99,4,74,R,Travel,Jun


,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name,month
3,19,94.0,2005-07-09 05:01:58,1.0,2.0,0.99,6,113,PG,Action,Jul
4,19,94.0,2005-07-30 08:02:39,1.0,1.0,0.99,6,113,PG,Action,Jul
8,19,95.0,2005-07-11 02:37:51,1.0,2.0,0.99,6,113,PG,Action,Jul
9,19,95.0,2005-07-28 07:21:55,1.0,1.0,0.99,6,113,PG,Action,Jul
14,19,97.0,2005-07-05 22:59:53,2.0,1.0,0.99,6,113,PG,Action,Jul
...,...,...,...,...,...,...,...,...,...,...,...
15892,989,4533.0,2005-07-12 20:58:04,1.0,2.0,4.99,4,74,R,Travel,Jul
15893,989,4533.0,2005-07-29 04:49:26,1.0,2.0,4.99,4,74,R,Travel,Jul
15897,989,4534.0,2005-07-12 01:15:24,2.0,2.0,4.99,4,74,R,Travel,Jul
15902,989,4535.0,2005-07-07 06:38:31,2.0,1.0,4.99,4,74,R,Travel,Jul


,film_id,inventory_id,rental_date,store_id,staff_id,rental_rate,rental_duration,length,rating,name,month
0,19,93.0,2005-08-02 01:16:59,1.0,2.0,0.99,6,113,PG,Action,Aug
1,19,93.0,2005-08-18 04:26:54,1.0,2.0,0.99,6,113,PG,Action,Aug
5,19,94.0,2005-08-21 17:51:06,1.0,1.0,0.99,6,113,PG,Action,Aug
10,19,95.0,2005-08-21 07:44:32,1.0,1.0,0.99,6,113,PG,Action,Aug
11,19,96.0,2005-08-01 11:25:28,1.0,2.0,0.99,6,113,PG,Action,Aug
...,...,...,...,...,...,...,...,...,...,...,...
15891,989,4532.0,2005-08-21 01:33:32,1.0,2.0,4.99,4,74,R,Travel,Aug
15894,989,4533.0,2005-08-21 15:00:49,1.0,1.0,4.99,4,74,R,Travel,Aug
15898,989,4534.0,2005-08-01 23:29:58,2.0,2.0,4.99,4,74,R,Travel,Aug
15899,989,4534.0,2005-08-22 10:19:58,2.0,1.0,4.99,4,74,R,Travel,Aug


create film june

In [134]:
rentalst=pd.crosstab(rental_may.film_id,rental_may.store_id)
rentalem=pd.crosstab(rental_may.film_id,rental_may.staff_id)
rental_y=pd.crosstab(rental_jun.film_id,rental_jun.month)
rentalst=rentalst.rename(columns={1:'rstore_1',2:'rstore_2'})
rentalem=rentalem.rename(columns={1:'employee_1',2:'employee_2'})
rental_y=rental_y.rename(columns={'Jun':'y'})

film_jun=pd.merge(left=film2,
                right=rentalst,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_jun=pd.merge(left=film_jun,
                right=rentalem,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_jun=pd.merge(left=film_jun,
                right=rental_y,
                how='left',
                left_on='film_id',
                right_on='film_id')

film_jun=film_jun.fillna(0)
film_jun=film_jun.drop('film_id',axis=1)
display(film_jun)


,rental_rate,rental_duration,length,rating,name,istore_1,istore_2,rstore_1,rstore_2,employee_1,employee_2,y
0,0.99,6,113,PG,Action,4.0,2.0,1.0,0.0,0.0,1.0,4.0
1,4.99,3,129,R,Action,2.0,4.0,2.0,0.0,1.0,1.0,3.0
2,2.99,5,168,NC-17,Action,2.0,0.0,2.0,0.0,0.0,2.0,2.0
3,0.99,6,68,NC-17,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.99,6,129,G,Action,3.0,2.0,1.0,0.0,0.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.99,7,48,PG-13,Travel,0.0,3.0,0.0,2.0,0.0,2.0,2.0
996,2.99,3,85,R,Travel,0.0,3.0,0.0,0.0,0.0,0.0,3.0
997,0.99,7,55,NC-17,Travel,3.0,3.0,0.0,0.0,0.0,0.0,4.0
998,2.99,7,139,R,Travel,3.0,2.0,0.0,0.0,0.0,0.0,2.0


create film July

In [133]:
rentalst=pd.crosstab(rental_jun.film_id,rental_jun.store_id)
rentalem=pd.crosstab(rental_jun.film_id,rental_jun.staff_id)
rental_y=pd.crosstab(rental_jul.film_id,rental_jul.month)
rentalst=rentalst.rename(columns={1:'rstore_1',2:'rstore_2'})
rentalem=rentalem.rename(columns={1:'employee_1',2:'employee_2'})
rental_y=rental_y.rename(columns={'Jul':'y'})

film_jul=pd.merge(left=film2,
                right=rentalst,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_jul=pd.merge(left=film_jul,
                right=rentalem,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_jul=pd.merge(left=film_jul,
                right=rental_y,
                how='left',
                left_on='film_id',
                right_on='film_id')

film_jul=film_jul.fillna(0)
film_jul=film_jul.drop('film_id',axis=1)
display(film_jul)

,rental_rate,rental_duration,length,rating,name,istore_1,istore_2,rstore_1,rstore_2,employee_1,employee_2,y
0,0.99,6,113,PG,Action,4.0,2.0,2.0,2.0,4.0,0.0,8.0
1,4.99,3,129,R,Action,2.0,4.0,2.0,1.0,2.0,1.0,8.0
2,2.99,5,168,NC-17,Action,2.0,0.0,2.0,0.0,2.0,0.0,4.0
3,0.99,6,68,NC-17,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.99,6,129,G,Action,3.0,2.0,2.0,1.0,2.0,1.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.99,7,48,PG-13,Travel,0.0,3.0,0.0,2.0,0.0,2.0,3.0
996,2.99,3,85,R,Travel,0.0,3.0,0.0,3.0,1.0,2.0,6.0
997,0.99,7,55,NC-17,Travel,3.0,3.0,2.0,2.0,2.0,2.0,9.0
998,2.99,7,139,R,Travel,3.0,2.0,1.0,1.0,1.0,1.0,6.0


create film  Aug

In [136]:
rentalst=pd.crosstab(rental_jul.film_id,rental_jul.store_id)
rentalem=pd.crosstab(rental_jul.film_id,rental_jul.staff_id)
rental_y=pd.crosstab(rental_aug.film_id,rental_aug.month)
rentalst=rentalst.rename(columns={1:'rstore_1',2:'rstore_2'})
rentalem=rentalem.rename(columns={1:'employee_1',2:'employee_2'})
rental_y=rental_y.rename(columns={'Aug':'y'})

film_aug=pd.merge(left=film2,
                right=rentalst,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_aug=pd.merge(left=film_aug,
                right=rentalem,
                how='left',
                left_on='film_id',
                right_on='film_id')
film_aug=pd.merge(left=film_aug,
                right=rental_y,
                how='left',
                left_on='film_id',
                right_on='film_id')

film_aug=film_aug.fillna(0)
film_aug=film_aug.drop('film_id',axis=1)
display(film_aug)

,rental_rate,rental_duration,length,rating,name,istore_1,istore_2,rstore_1,rstore_2,employee_1,employee_2,y
0,0.99,6,113,PG,Action,4.0,2.0,4.0,4.0,4.0,4.0,7.0
1,4.99,3,129,R,Action,2.0,4.0,3.0,5.0,4.0,4.0,8.0
2,2.99,5,168,NC-17,Action,2.0,0.0,4.0,0.0,2.0,2.0,2.0
3,0.99,6,68,NC-17,Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.99,6,129,G,Action,3.0,2.0,5.0,3.0,3.0,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.99,7,48,PG-13,Travel,0.0,3.0,0.0,3.0,3.0,0.0,5.0
996,2.99,3,85,R,Travel,0.0,3.0,0.0,6.0,4.0,2.0,2.0
997,0.99,7,55,NC-17,Travel,3.0,3.0,4.0,5.0,3.0,6.0,8.0
998,2.99,7,139,R,Travel,3.0,2.0,2.0,4.0,3.0,3.0,7.0
